# Build a Multimodal AI Shopping Agent with Voyage AI and Pixeltable

**Best-in-class embeddings and rerankers powering an intelligent product assistant**

Modern e-commerce platforms need more than keyword search—they need AI that understands customer intent. Queries like "comfortable shoes for standing all day" or "gift ideas for a tech enthusiast" require semantic understanding, not string matching.

In this tutorial, we'll build an **AI-powered shopping agent** that combines:

- **[Voyage AI](https://voyageai.com)**: State-of-the-art embedding models (voyage-3.5) and rerankers (rerank-2.5) purpose-built for search and retrieval
- **[Pixeltable](https://pixeltable.com)**: Declarative AI data infrastructure for embeddings, tool calling, and agentic pipelines

Using real Amazon product data, we'll create:

1. 🔍 **Semantic Product Search**: Multi-column embeddings with weighted similarity
2. 🎯 **Two-Stage Retrieval**: Fast embedding search + precise reranking
3. 📊 **Declarative Pipelines**: Multiple search strategies as computed columns
4. 🤖 **AI Shopping Agent**: LLM that orchestrates search and lookup tools

### Prerequisites

- A Voyage AI API key ([get one free](https://www.voyageai.com/))
- An OpenAI API key (for the agent)
- Basic familiarity with Python


## Setup

First, let's install the required packages and configure our environment.


In [1]:
%pip install -qU pixeltable voyageai openai jinja2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fiftyone 1.7.1 requires pymongo~=4.9.2, but you have pymongo 4.16.0 which is incompatible.
fiftyone 1.7.1 requires sse-starlette<1,>=0.10.3, but you have sse-starlette 2.3.6 which is incompatible.
lightning 2.5.0.post0 requires fsspec[http]<2026.0,>=2022.5.0, but you have fsspec 2026.1.0 which is incompatible.
lightning 2.5.0.post0 requires packaging<25.0,>=20.0, but you have packaging 25.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import getpass

if 'VOYAGE_API_KEY' not in os.environ:
    os.environ['VOYAGE_API_KEY'] = getpass.getpass('Enter your Voyage AI API key: ')

In [3]:
import pixeltable as pxt
from pixeltable.functions import voyageai

# Create a fresh workspace for this demo
pxt.drop_dir('ecommerce_search', force=True)
pxt.create_dir('ecommerce_search')

/opt/miniconda3/envs/pixeltable/lib/python3.11/site-packages/pixeltable/env.py:492: UserWarning: Progress reporting is disabled because ipywidgets is not installed. To fix this, run: `pip install ipywidgets`
  warnings.warn(


Connected to Pixeltable database at: postgresql+psycopg://postgres:@/pixeltable?host=/Users/pjlb/.pixeltable/pgdata
Created directory 'ecommerce_search'.


## Load Amazon Product Data

We'll use a pre-processed subset of the [Amazon Product Dataset 2020](https://huggingface.co/datasets/calmgoose/amazon-product-data-2020), which contains real product listings with rich metadata including:

- Product names and descriptions
- Categories and specifications
- Pricing information
- **One image URL per row** (the original dataset had multiple images pipe-separated; we've split them for easier processing)

The dataset contains ~1,800 rows from 500 products, with each product having 1-7 images.


In [4]:
# Pre-processed Amazon product dataset URL
# Note: Update URL to pixeltable/pixeltable after PR is merged
DATASET_URL = 'https://raw.githubusercontent.com/pierrebrunelle/pixeltable/feature/voyageai-ecommerce-search-notebook/docs/resources/amazon_products_with_images.parquet'

In [5]:
# Preview: load with pandas to see the data
import pandas as pd
df = pd.read_parquet(DATASET_URL)

### Import into Pixeltable

Now let's import this dataset into Pixeltable. Pixeltable can directly ingest PyArrow tables via `create_table(source=...)`.


In [ ]:
# Import the DataFrame into Pixeltable
# schema_overrides: explicitly type string columns and treat Image URLs as images
# on_error='ignore' skips rows where image URLs return 404 (some Amazon URLs expire)
products = pxt.create_table(
    'ecommerce_search.products',
    source=df,
    schema_overrides={
        'Uniq_Id': pxt.String,
        'Product_Name': pxt.String,
        'Category': pxt.String,
        'Selling_Price': pxt.String,
        'About_Product': pxt.String,
        'Image': pxt.Image,
    },
    on_error='ignore'
)

products.head(3)

Error: Could not infer Pixeltable type of column: Uniq_Id (dtype: str)

## Multi-Column Embedding Strategy

Instead of combining all product fields into a single text, we'll create **separate embedding indexes** for each searchable column. This approach offers several advantages:

- **Flexible weighting**: Combine results from different columns with custom weights
- **Column-specific queries**: Search only product names, or only descriptions
- **Better relevance**: Each embedding captures the semantic meaning of its specific field


In [ ]:
# Define the embedding function once for reuse
# The .using() syntax fixes the model parameter, creating a specialized embedding function
embed_fn = voyageai.embeddings.using(model='voyage-3.5', input_type='document')

# Add embedding indexes for each searchable text column
products.add_embedding_index('Product_Name', embedding=embed_fn)
products.add_embedding_index('Category', embedding=embed_fn)
products.add_embedding_index('About_Product', embedding=embed_fn)

In [ ]:
# View the table structure - note the embedding indexes
products

table 'ecommerce_search.products'

    Column Name    Type Computed With
        Uniq_Id  String              
   Product_Name  String              
       Category  String              
  Selling_Price  String              
  About_Product  String              
          Image   Image              
      image_idx     Int              

 Index Name         Column  Metric                                          Embedding
       idx7   Product_Name  cosine  embeddings(Product_Name, model='voyage-3.5', i...
       idx8       Category  cosine  embeddings(Category, model='voyage-3.5', input...
       idx9  About_Product  cosine  embeddings(About_Product, model='voyage-3.5', ...

## Semantic Product Search with Query Functions

With embedding indexes on multiple columns, we can create **query functions** that combine similarity scores with configurable weights. Query functions (`@pxt.query`) are declarative—they can be used as computed columns that execute automatically when data is inserted.


In [ ]:
@pxt.query
def weighted_search(
    query_text: str, 
    limit: int = 5,
    name_weight: float = 0.4,
    category_weight: float = 0.2,
    description_weight: float = 0.4
):
    """
    Search products using weighted similarity across multiple columns.
    
    As a @pxt.query function, this can be:
    - Called directly: weighted_search('toys', limit=10)
    - Used in computed columns with different weight configurations
    """
    name_sim = products['Product_Name'].similarity(string=query_text)
    category_sim = products['Category'].similarity(string=query_text)
    description_sim = products['About_Product'].similarity(string=query_text)
    
    combined_score = (
        name_weight * name_sim + 
        category_weight * category_sim + 
        description_weight * description_sim
    )
    
    return (
        products
        .order_by(combined_score, asc=False)
        .limit(limit)
        .select(
            products['Product_Name'],
            products['Category'],
            products['Selling_Price'],
            score=combined_score
        )
    )


### Direct Query Execution

Query functions can be called directly for interactive exploration:


In [ ]:
# Natural language query with default weights
weighted_search("fun games for kids birthday party")


weighted_search('fun games for kids birthday party')

In [ ]:
# Adjust weights to prioritize product names
weighted_search("educational toys", name_weight=0.7, category_weight=0.1, description_weight=0.2)

weighted_search('educational toys', name_weight=0.7, category_weight=0.1, description_weight=0.2)

### Declarative Search with Computed Columns

The real power of `@pxt.query` functions is using them as **computed columns**. We can create a searches table with multiple weighting strategies—each computed automatically when queries are inserted:

In [ ]:
# Create a searches table with multiple weighting strategies as computed columns
searches = pxt.create_table(
    'ecommerce_search.searches',
    {'query': pxt.String}
)

# Strategy 1: Balanced (default weights - good general-purpose search)
searches.add_computed_column(
    balanced=weighted_search(searches.query)
)

# Strategy 2: Name-focused (prioritizes exact product name matches)
searches.add_computed_column(
    name_focused=weighted_search(
        searches.query,
        name_weight=0.7, category_weight=0.1, description_weight=0.2
    )
)

# Strategy 3: Description-focused (finds products by feature descriptions)
searches.add_computed_column(
    description_focused=weighted_search(
        searches.query,
        name_weight=0.2, category_weight=0.1, description_weight=0.7
    )
)

Created table 'searches'.
Added 0 column values with 0 errors in 0.01 s
Added 0 column values with 0 errors in 0.01 s
Added 0 column values with 0 errors in 0.01 s


No rows affected.

In [ ]:
# Insert a query - all three strategies compute automatically!
searches.insert([{'query': 'durable outdoor toys for active kids'}])

Inserted 1 row with 0 errors in 1.36 s (0.73 rows/s)


1 row inserted.

In [ ]:
# Compare results: Balanced strategy
searches.select(searches.query, searches.balanced).collect()

query,balanced
durable outdoor toys for active kids,"[{""score"": null, ""Category"": ""Toys & Games | Puzzles"", ""Product_Name"": ""Awesome Paw Patrol Bundle of 3 Pieces"", ""Selling_Price"": ""\$8.52""}, {""score"": null, ""Category"": ""Clothing, Shoes & Jewelry | Costumes & Accessories | Women | Costumes & Cosplay Apparel | Costumes"", ""Product_Name"": ""Rubie's Costume Womens Police Costume"", ""Selling_Price"": ""\$16.03 - \$24.79""}, {""score"": null, ""Category"": ""Sports & Outdoors | Sports & Fitness | Leisure Sports & Game Room | Outdoor Games & Activities | Balls | Playground Balls"", ""Product_Name"": ""American Educational Vinyl Clever Catch Elementary Science Insects Ball, 24\"" Diameter"", ""Selling_Price"": ""\$17.40""}, {""score"": null, ""Category"": """", ""Product_Name"": ""Gowi Toys Austria Police Van with Police Play Figures"", ""Selling_Price"": ""\$11.16""}, {""score"": null, ""Category"": """", ""Product_Name"": ""Disney Rapunzel Acrylic Key Ring, Multicolor"", ""Selling_Price"": ""\$3.33""}]"


In [ ]:
# Compare results: Name-focused strategy (different top results!)
searches.select(searches.query, searches.name_focused).collect()

query,name_focused
durable outdoor toys for active kids,"[{""score"": null, ""Category"": ""Toys & Games | Puzzles"", ""Product_Name"": ""Awesome Paw Patrol Bundle of 3 Pieces"", ""Selling_Price"": ""\$8.52""}, {""score"": null, ""Category"": ""Clothing, Shoes & Jewelry | Costumes & Accessories | Women | Costumes & Cosplay Apparel | Costumes"", ""Product_Name"": ""Rubie's Costume Womens Police Costume"", ""Selling_Price"": ""\$16.03 - \$24.79""}, {""score"": null, ""Category"": ""Sports & Outdoors | Sports & Fitness | Leisure Sports & Game Room | Outdoor Games & Activities | Balls | Playground Balls"", ""Product_Name"": ""American Educational Vinyl Clever Catch Elementary Science Insects Ball, 24\"" Diameter"", ""Selling_Price"": ""\$17.40""}, {""score"": null, ""Category"": """", ""Product_Name"": ""Gowi Toys Austria Police Van with Police Play Figures"", ""Selling_Price"": ""\$11.16""}, {""score"": null, ""Category"": """", ""Product_Name"": ""Disney Rapunzel Acrylic Key Ring, Multicolor"", ""Selling_Price"": ""\$3.33""}]"


## Boost Relevance with Voyage AI Reranking

While semantic search is powerful, we can further improve result quality using Voyage AI's **rerank-2.5** model. The two-stage retrieval pattern:

1. **First stage**: Use embeddings to quickly retrieve candidates (top 15)
2. **Second stage**: Use the reranker to precisely score and reorder results

Let's add reranking as another computed column to our searches table:


In [ ]:
# First, we need candidates for reranking (reuse weighted_search with more results)
searches.add_computed_column(
    candidates=weighted_search(searches.query, limit=15)
)

# Add reranking using Voyage AI's rerank-2.5 model
# Reranks the embedding search results for improved precision
searches.add_computed_column(
    reranked=voyageai.rerank(
        searches.query,
        searches.candidates['About_Product'],  # Rerank based on descriptions
        model='rerank-2.5',
        top_k=5
    )
)


Added 1 column value with 0 errors in 0.60 s (1.68 rows/s)
Added 1 column value with 0 errors in 0.01 s (73.87 rows/s)


1 row updated.

In [ ]:
# View the complete searches table - now with 5 computed columns!
# For each query: 3 weighting strategies + candidates + reranked results
searches


table 'ecommerce_search.searches'

          Column Name    Type                                      Computed With
                query  String                                                   
             balanced    Json                             weighted_search(query)
         name_focused    Json  weighted_search(query, name_weight=0.7, catego...
  description_focused    Json  weighted_search(query, name_weight=0.2, catego...
           candidates    Json                   weighted_search(query, limit=15)
             reranked    Json  rerank(query, candidates.About_Product, model=...

In [ ]:
# The query we inserted earlier now has reranked results too!
# Compare: balanced embedding search vs reranked
searches.select(
    searches.query,
    searches.balanced,
    searches.reranked
).collect()

query,balanced,reranked
durable outdoor toys for active kids,"[{""score"": null, ""Category"": ""Toys & Games | Puzzles"", ""Product_Name"": ""Awesome Paw Patrol Bundle of 3 Pieces"", ""Selling_Price"": ""\$8.52""}, {""score"": null, ""Category"": ""Clothing, Shoes & Jewelry | Costumes & Accessories | Women | Costumes & Cosplay Apparel | Costumes"", ""Product_Name"": ""Rubie's Costume Womens Police Costume"", ""Selling_Price"": ""\$16.03 - \$24.79""}, {""score"": null, ""Category"": ""Sports & Outdoors | Sports & Fitness | Leisure Sports & Game Room | Outdoor Games & Activities | Balls | Playground Balls"", ""Product_Name"": ""American Educational Vinyl Clever Catch Elementary Science Insects Ball, 24\"" Diameter"", ""Selling_Price"": ""\$17.40""}, {""score"": null, ""Category"": """", ""Product_Name"": ""Gowi Toys Austria Police Van with Police Play Figures"", ""Selling_Price"": ""\$11.16""}, {""score"": null, ""Category"": """", ""Product_Name"": ""Disney Rapunzel Acrylic Key Ring, Multicolor"", ""Selling_Price"": ""\$3.33""}]",None


In [ ]:
# Insert another query to see the full pipeline in action
searches.insert([{'query': 'safe educational toys for toddlers'}])


Inserted 1 row with 0 errors in 1.78 s (0.56 rows/s)


1 row inserted.

### Compare Embedding Search vs. Reranked Results

The reranker often surfaces more relevant results by considering the full query-document relationship:


In [ ]:
# View embedding results (balanced strategy) vs reranked results
searches.select(
    searches.query,
    embedding_top_5=searches.balanced,
    reranked_top_5=searches.reranked['results']
).where(searches.query == 'safe educational toys for toddlers').collect()


query,embedding_top_5,reranked_top_5
safe educational toys for toddlers,"[{""score"": null, ""Category"": ""Toys & Games | Puzzles"", ""Product_Name"": ""Awesome Paw Patrol Bundle of 3 Pieces"", ""Selling_Price"": ""\$8.52""}, {""score"": null, ""Category"": ""Clothing, Shoes & Jewelry | Costumes & Accessories | Women | Costumes & Cosplay Apparel | Costumes"", ""Product_Name"": ""Rubie's Costume Womens Police Costume"", ""Selling_Price"": ""\$16.03 - \$24.79""}, {""score"": null, ""Category"": ""Sports & Outdoors | Sports & Fitness | Leisure Sports & Game Room | Outdoor Games & Activities | Balls | Playground Balls"", ""Product_Name"": ""American Educational Vinyl Clever Catch Elementary Science Insects Ball, 24\"" Diameter"", ""Selling_Price"": ""\$17.40""}, {""score"": null, ""Category"": """", ""Product_Name"": ""Gowi Toys Austria Police Van with Police Play Figures"", ""Selling_Price"": ""\$11.16""}, {""score"": null, ""Category"": """", ""Product_Name"": ""Disney Rapunzel Acrylic Key Ring, Multicolor"", ""Selling_Price"": ""\$3.33""}]",None


In [ ]:
# View all queries and their reranked results
searches.select(searches.query, searches.reranked).collect()


query,reranked
durable outdoor toys for active kids,None
safe educational toys for toddlers,None


## Incremental Updates: Adding New Products

One of Pixeltable's key strengths is handling incremental updates. When new products are added to the catalog, embeddings are computed automatically—no need to reprocess the entire dataset.


In [ ]:
# Add new products - embeddings for all three indexes are computed automatically!
new_products = [
    {
        'Uniq_Id': 'new_001',
        'Product_Name': 'Ultimate STEM Building Kit - 500 Pieces',
        'Category': 'Toys & Games | Building Toys | Building Sets',
        'About_Product': 'Educational building set with 500 pieces for ages 6+. Includes gears, motors, and instruction booklet for 50 projects. Develops problem-solving and engineering skills.',
        'Selling_Price': '$49.99',
        'Image': None,  # Use None for no image, not empty string
        'image_idx': 0
    },
    {
        'Uniq_Id': 'new_002', 
        'Product_Name': 'Outdoor Adventure Binoculars for Kids',
        'Category': 'Toys & Games | Sports & Outdoor Play | Exploration Toys',
        'About_Product': 'Kid-friendly binoculars with 8x magnification, rubber grip, and neck strap. Perfect for bird watching, camping, and nature exploration. Shockproof design.',
        'Selling_Price': '$24.99',
        'Image': None,  # Use None for no image, not empty string
        'image_idx': 0
    }
]

products.insert(new_products)


Inserted 2 rows with 0 errors in 0.35 s (5.72 rows/s)


2 rows inserted.

In [ ]:
# Search should now find the new products
weighted_search("STEM toys for kids who like to build things")


weighted_search('STEM toys for kids who like to build things')

## Agentic Search: LLM-Powered Product Assistant

Now let's combine everything into an **agentic pipeline** where an LLM decides which tools to use:

- **Semantic search** (`weighted_search`): Find products by meaning
- **Exact lookup** (`get_product_by_id`): Get specific product details by ID

The LLM orchestrates these tools to answer complex questions like *"Tell me about product new_001 and find similar educational toys"*.

In [ ]:
# Create an exact product lookup using retrieval_udf
# This queries by product ID for precise lookups
get_product_by_id = pxt.retrieval_udf(
    products,
    name='get_product_by_id',
    description='Look up a specific product by its unique ID (Uniq_Id)',
    parameters=['Uniq_Id'],
    limit=1
)

In [ ]:
# Wrap weighted_search as a UDF for tool calling
# The LLM will call this for semantic product search
@pxt.udf
def search_products_tool(query: str) -> str:
    """Search for products using semantic similarity. Use this when the user 
    wants to find products by description, features, or category."""
    results = weighted_search(query, limit=5)
    # Format results as a string for LLM consumption
    output = []
    for row in results:
        output.append(f"- {row['Product_Name']} ({row['Selling_Price']}) - {row['Category']}")
    return "\n".join(output) if output else "No products found."

In [ ]:
# Bundle both tools for LLM use
product_tools = pxt.tools(search_products_tool, get_product_by_id)

In [ ]:
# Set up OpenAI for the agent (or use Anthropic, etc.)
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter your OpenAI API key: ')

from pixeltable.functions import openai

In [ ]:
# Create the agent table with tool-calling pipeline
agent = pxt.create_table(
    'ecommerce_search.agent',
    {'question': pxt.String}
)

# System prompt for the product assistant
SYSTEM_PROMPT = """You are a helpful e-commerce product assistant. You have access to two tools:
1. search_products_tool: For finding products by description or features
2. get_product_by_id: For looking up specific products by their ID

Use these tools to answer customer questions about products. Be concise and helpful."""

# LLM decides which tools to call
agent.add_computed_column(
    llm_response=openai.chat_completions(
        messages=[
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': agent.question}
        ],
        model='gpt-4o-mini',
        tools=product_tools
    )
)

# Automatically execute the tool calls
agent.add_computed_column(
    tool_results=openai.invoke_tools(product_tools, agent.llm_response)
)

In [ ]:
# UDF to assemble the final prompt with tool results
@pxt.udf
def assemble_answer_prompt(question: str, tool_results: dict) -> list[dict]:
    """Combine the original question and tool results into a prompt for final answer."""
    # Format tool results for the LLM
    results_text = []
    for tool_name, outputs in (tool_results or {}).items():
        for output in outputs:
            results_text.append(f"[{tool_name}]: {output}")
    
    context = "\n".join(results_text) if results_text else "No tool results available."
    
    return [
        {'role': 'system', 'content': 'You are a helpful product assistant. Use the tool results to answer the question concisely.'},
        {'role': 'user', 'content': f"Question: {question}\n\nTool Results:\n{context}\n\nPlease provide a helpful answer based on the tool results."}
    ]

# Generate final answer using tool results
agent.add_computed_column(
    answer_prompt=assemble_answer_prompt(agent.question, agent.tool_results)
)

agent.add_computed_column(
    answer=openai.chat_completions(
        messages=agent.answer_prompt,
        model='gpt-4o-mini'
    )['choices'][0]['message']['content']
)

In [ ]:
# Test the agent with different types of questions
test_questions = [
    {'question': 'What educational toys do you have for kids who like building?'},
    {'question': 'Tell me about product new_001'},
    {'question': 'I need outdoor toys for active children. What do you recommend?'},
]

agent.insert(test_questions)

In [ ]:
# View the agent's answers
agent.select(agent.question, agent.answer).collect()

In [ ]:
# See which tools the LLM called for each question
agent.select(
    agent.question,
    agent.tool_results,
    agent.answer
).collect()

## Summary

In this tutorial, we built a production-ready semantic search system for e-commerce by combining:

### Voyage AI Features
- **voyage-3.5 Embeddings**: State-of-the-art embedding model for semantic search across product names, categories, and descriptions
- **rerank-2.5 Reranker**: Two-stage retrieval pattern that combines fast embedding search with precise cross-encoder reranking

### Pixeltable Capabilities
- **Multi-Column Embedding Indexes**: Separate indexes per field with `add_embedding_index()`
- **Query Functions (`@pxt.query`)**: Reusable, parameterized search logic with weighted similarity
- **Computed Columns**: Multiple weighting strategies computed automatically
- **Retrieval UDFs (`pxt.retrieval_udf`)**: Exact lookups by key (product ID, SKU, etc.)
- **Tool Calling (`pxt.tools`)**: Bundle search and lookup as tools for LLM agents
- **Agentic Pipelines**: LLM decides which tools to call, results combined via UDFs

### Key Takeaways
1. **Semantic Search > Keyword Search**: Find "comfortable shoes for standing" even if products don't contain those exact words
2. **Query Functions as Building Blocks**: Define once, use in multiple computed columns or as tools
3. **Two-Stage Retrieval**: Embeddings for fast candidate retrieval, reranker for precision
4. **Agentic Architecture**: Combine semantic search + exact lookup + LLM reasoning in one pipeline
5. **Declarative Everything**: Insert a row → tools called, results combined, answer generated automatically

This architecture adapts easily to other use cases like document search, support ticket routing, or recommendation systems.


## Learn More

**Voyage AI Resources**
- [Voyage AI Documentation](https://docs.voyageai.com/)
- [Embedding Models](https://docs.voyageai.com/docs/embeddings) - voyage-3.5 and other models
- [Reranker Guide](https://docs.voyageai.com/docs/reranker) - rerank-2.5 and rerank-2.5-lite
- [Voyage AI + MongoDB](https://www.mongodb.com/blog/post/voyage-ai-joins-mongodb-to-advance-ai-powered-applications) - Voyage AI is now part of MongoDB

**Pixeltable Resources**
- [Documentation](https://docs.pixeltable.com/)
- [Embedding Indexes Guide](https://docs.pixeltable.com/platform/embedding-indexes)
- [Tool Calling with LLMs](https://docs.pixeltable.com/howto/cookbooks/agents/llm-tool-calling) - Agent patterns
- [Data Lookup UDFs](https://docs.pixeltable.com/howto/cookbooks/agents/pattern-data-lookup) - Retrieval UDFs

**Get Started**
- [Sign up for Voyage AI](https://www.voyageai.com/) (free tier available)
- [Install Pixeltable](https://github.com/pixeltable/pixeltable): `pip install pixeltable`